# TOOLS AND MCP CONNECTIONS

## 1.	WHAT is the difference between TOOLS and MCP CONNECTIONS?

A Tool is like a button and a MCP is like a USBC port

We use a tool to perform a specific function or task within a software application or system. Tools are typically designed to help users accomplish specific goals, such as editing text, creating graphics, or managing data. 



Examples of tools:
•	search_google(query)
•	get_employee(id)
•	send_email(to, subject, body)
•	run_sql(query)
•	schedule_meeting(time, people)

MCP (Model Context Protocol) is the standardized way AI discovers and talks to tools.
Without an MCP connection, if an AI like ChatGPT wanted to talk to gmail, it would need custom glue code to execute its goals. So lets say we wanted our AI agent to access data in LinkedIn. The company would need to create a MCP server connecton for AI's to talk to linkedIn. LinkedIn would then be able to control the access levels, data formats, and security protocols for AI's accessing its data. 

Examples of MCP connections:
•	Canvas MCP connection
•	Google Search MCP connection
•	SQL Database MCP connection
•	Calendar MCP connection


We will be using a Canvas MCP, which means we will need to establish a MCP connection and then use the MCP's tool to allow our agent to talk to canvas. 



2.	LETS USE THEM
User → AI → MCP Client → MCP Server → Tool → Result → AI → User

pre requisisites:





## Canvas



First install dependencies

In [ ]:
%pip install nest_asyncio
%pip install langchain
%pip install langchain-google-genai
%pip install langchain-mcp-adapters
%pip install nest_asyncio langchain langchain-google-genai langchain-mcp-adapters


In [5]:
import os
import asyncio
import nest_asyncio
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.agents import create_agent
from langchain_mcp_adapters.client import MultiServerMCPClient


In [ ]:

# 1. SETUP & FIX ASYNC (Required for Notebooks)
# This fixes "RuntimeError: This event loop is already running"
nest_asyncio.apply()

from dotenv import load_dotenv
load_dotenv()

# 2. SETUP CREDENTIALS
# Set CANVAS_API_TOKEN and GOOGLE_API_KEY (or GEMINI_API_KEY) as environment variables in your shell.
# You need your Canvas URL and Token (from your school profile settings)
os.environ["CANVAS_API_URL"] = ""
os.environ["CANVAS_API_TOKEN"] = ""
os.environ["GOOGLE_API_KEY"] = ""



### Load your API keys securely (Best Practice for Notebooks)
### use export GOOGLE_API_KEY='your_google_api_key_here' in your terminal before running the notebook
### Then use echo $GOOGLE_API_KEY to verify it's set


In [7]:

async def load_mcp_agent():
    llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0,
    api_key=os.environ.get("GOOGLE_API_KEY") or os.environ.get("GEMINI_API_KEY")
)

    mcp_client = MultiServerMCPClient({
    "canvas": {
        "command": "canvas-mcp-server",
        "args": [],
        "transport": "stdio"
    }
    })



    mcp_tools = await mcp_client.get_tools()
    all_tools = mcp_tools + [student_file_tool]


    agent = create_agent(
        llm,
        tools=all_tools,
        system_prompt="You are a helpful assistant for a student. Use the tools to assist with their requests."
    )

    return agent, mcp_client



In [8]:

from langchain_core.tools import Tool
from pprint import pprint

STUDENT_DATA_DIR = "student_data"

def read_student_files(query: str) -> str:
    """
    Reads all files in student_data/ and returns their contents.
    The query is ignored for now (simple version).
    """
    if not os.path.exists(STUDENT_DATA_DIR):
        return "student_data folder not found."

    all_text = []
    for fname in os.listdir(STUDENT_DATA_DIR):
        path = os.path.join(STUDENT_DATA_DIR, fname)

        if not os.path.isfile(path):
            continue

        try:
            with open(path, "r", encoding="utf-8", errors="ignore") as f:
                content = f.read()
            all_text.append(f"\n--- {fname} ---\n{content}")
        except Exception as e:
            all_text.append(f"\n--- {fname} ---\nERROR: {e}")

    return "\n".join(all_text)



# Defining tool

student_file_tool = Tool(
    name="student_data_reader",
    func=read_student_files,
    description="Use this to read files from the local student_data folder. Whenever the user wants to access local files, go here. User may call this folder 'my folder, the folder' or other general terms"
)





async def main():
    agent, mcp_client = await load_mcp_agent()

    user_prompt = "Go into my folder into the study guide and give me a 3 sentence summary of what I should study"

    result = await agent.ainvoke({
    "messages": [
        {"role": "user", "content": user_prompt}
    ]
})
    # Inspect full response to find the correct field to use
    pprint(result)



# Run the async main function in the notebook
asyncio.run(main())

Key 'additionalProperties' is not supported in schema, ignoring
Key 'additionalProperties' is not supported in schema, ignoring


{'messages': [HumanMessage(content='Go into my folder into the study guide and give me a 3 sentence summary of what I should study', additional_kwargs={}, response_metadata={}, id='98094fd7-0f49-421c-9790-8f695edf8100'),
              AIMessage(content='', additional_kwargs={'function_call': {'name': 'student_data_reader', 'arguments': '{"__arg1": "student_data/study_guide.txt"}'}, '__gemini_function_call_thought_signatures__': {'27362cc6-7bc5-4f60-83a3-89e82e3e5a39': 'Cs8FAb4+9vs0Ff5UGyRGRjbTKOj6jYyjPu3JuWZ2wvF+11/I8kiHP0gh282p6niyXJ1/SYBgAAIVj1G5ule4uKkXvnrMj770wnwWgP0TIA1xECHsIMbhRoFEJVIkIQjoJmzHKmyYMd/BJeEAs6MGtMbk0NJYjM7aiD+GHw3zxj+WoWGQmBu+Iu5owdWEPrpxkBh8jnL6qITJbEjsd6Y32Lx83YbyjrJ1qAOO/6cEtPObHpUZbEAdYeANz8rMY0O4U5N2vJMINQNAy/sNWtc3vU4P4DXh5/PehUrvamgiIxb14yAtP3AE6FXir6XrbLG3PUDBG5EjGITGqIvL9ECTLJmuCiGig0ii8TVzgRjguXTPfMkYXJbkLfwN1/fftGhSD/i8P5aTcjTO/GLu3VNSJyz+gurhAeV7Il1GfHSrcdpHdgNA7n1nxQYc4xEgjz6ObBlKukHRK7aqQi6cpqVvarc8f1gC7LnGZAxz/l+w0Ocvxe+b96mawYHAUSLh26uHv/4Ztw5lGhzzXE